In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect

In [3]:
import numpy as np
import pandas as pd

In [4]:
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [5]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [6]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [7]:
# reflect an existing database into a new model
Base = automap_base()
Base.prepare(engine, reflect=True)

# reflect the tables
Base.classes.keys()

['measurement', 'station']

In [8]:
 inspector = inspect(engine)
table_list=inspector.get_table_names()
table_list

['measurement', 'station']

In [57]:
print("--------------------------")
for table in table_list:
    columns = inspector.get_columns(table)
    print(f'Table name:     {table}\n')
    for c in columns:
        print(c['name'],c["type"])
    print("--------------------------")

--------------------------
Table name:     measurement

id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT
--------------------------
Table name:     station

id INTEGER
station TEXT
name TEXT
latitude FLOAT
longitude FLOAT
elevation FLOAT
--------------------------


In [64]:
# Map Station class
STAT = Base.classes.station

# Map Measurement class
MEASUR = Base.classes.measurement

In [59]:
session = Session(bind=engine)

In [65]:
 session.query(STAT.station, STAT.name, STAT.elevation).all()

[('USC00519397', 'WAIKIKI 717.2, HI US', 3.0),
 ('USC00513117', 'KANEOHE 838.1, HI US', 14.6),
 ('USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 7.0),
 ('USC00517948', 'PEARL CITY, HI US', 11.9),
 ('USC00518838', 'UPPER WAHIAWA 874.3, HI US', 306.6),
 ('USC00519523', 'WAIMANALO EXPERIMENTAL FARM, HI US', 19.5),
 ('USC00519281', 'WAIHEE 837.5, HI US', 32.9),
 ('USC00511918', 'HONOLULU OBSERVATORY 702.2, HI US', 0.9),
 ('USC00516128', 'MANOA LYON ARBO 785.2, HI US', 152.4)]

In [78]:
# what is tobs?
 session.query(STAT.station, MEASUR.station, MEASUR.tobs, MEASUR.prcp,MEASUR.date).all()

[('USC00519397', 'USC00519397', 65.0, 0.08, '2010-01-01'),
 ('USC00519397', 'USC00519397', 63.0, 0.0, '2010-01-02'),
 ('USC00519397', 'USC00519397', 74.0, 0.0, '2010-01-03'),
 ('USC00519397', 'USC00519397', 76.0, 0.0, '2010-01-04'),
 ('USC00519397', 'USC00519397', 73.0, None, '2010-01-06'),
 ('USC00519397', 'USC00519397', 70.0, 0.06, '2010-01-07'),
 ('USC00519397', 'USC00519397', 64.0, 0.0, '2010-01-08'),
 ('USC00519397', 'USC00519397', 68.0, 0.0, '2010-01-09'),
 ('USC00519397', 'USC00519397', 73.0, 0.0, '2010-01-10'),
 ('USC00519397', 'USC00519397', 64.0, 0.01, '2010-01-11'),
 ('USC00519397', 'USC00519397', 61.0, 0.0, '2010-01-12'),
 ('USC00519397', 'USC00519397', 66.0, 0.0, '2010-01-14'),
 ('USC00519397', 'USC00519397', 65.0, 0.0, '2010-01-15'),
 ('USC00519397', 'USC00519397', 68.0, 0.0, '2010-01-16'),
 ('USC00519397', 'USC00519397', 64.0, 0.0, '2010-01-17'),
 ('USC00519397', 'USC00519397', 72.0, 0.0, '2010-01-18'),
 ('USC00519397', 'USC00519397', 66.0, 0.0, '2010-01-19'),
 ('USC0051

In [81]:
session.query(MEASUR.date).order_by(MEASUR.date.desc()).first()

('2017-08-23')

In [80]:
session.query(MEASUR.date).order_by(MEASUR.date).first()

('2010-01-01')

In [69]:
date_count =  session.query(MEASUR.date).count()
station_countS= session.query(STAT.station).count()
pecip_count = session.query(MEASUR.prcp).count()
station_countM= session.query(MEASUR.station).count()

print(f'Number of Stations (table s): {station_countS}\nNumber of Stations (table m): {station_countM}\nNumber of Dates: {date_count}\nNumber of Preciptation observations {pecip_count}')

Number of Stations (table s): 9
Number of Stations (table m): 19550
Number of Dates: 19550
Number of Preciptation observations 19550


# Exploratory Climate Analysis

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results

# Calculate the date 1 year ago from the last data point in the database

# Perform a query to retrieve the data and precipitation scores

# Save the query results as a Pandas DataFrame and set the index to the date column

# Sort the dataframe by date

# Use Pandas Plotting with Matplotlib to plot the data


In [87]:
#precipitation is from a Range of 2017 -08-23 to 2010-01-01
# Data used will be from 2016-08-23

# Design a query to retrieve the last 12 months of precipitation data and plot the results
precip_12months = session.query(MEASUR).\
    filter(MEASUR.date > "2016-08-23").count()
precip_12months

2223

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data

In [ ]:
# Design a query to show how many stations are available in this dataset?


In [ ]:
# What are the most active stations? (i.e. what stations have the most rows)?
# List the stations and the counts in descending order.


In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature of the most active station?


In [ ]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


## Bonus Challenge Assignment

In [ ]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# function usage example
print(calc_temps('2012-02-28', '2012-03-05'))

In [ ]:
# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax 
# for your trip using the previous year's data for those same dates.


In [ ]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for the y value
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)


In [ ]:
# Calculate the total amount of rainfall per weather station for your trip dates using the previous year's matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation



In [ ]:
# Create a query that will calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)

def daily_normals(date):
    """Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
    """
    
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    return session.query(*sel).filter(func.strftime("%m-%d", Measurement.date) == date).all()
    
daily_normals("01-01")

In [ ]:
# calculate the daily normals for your trip
# push each tuple of calculations into a list called `normals`

# Set the start and end date of the trip

# Use the start and end date to create a range of dates

# Stip off the year and save a list of %m-%d strings

# Loop through the list of %m-%d strings and calculate the normals for each date


In [ ]:
# Load the previous query results into a Pandas DataFrame and add the `trip_dates` range as the `date` index


In [ ]:
# Plot the daily normals as an area plot with `stacked=False`
